# Week 2: Word Embedding
## [IMDB dataset for binary sentiment classification](http://ai.stanford.edu/~amaas/data/sentiment/)

In [3]:
### Import modules:

import tensorflow as tf 
print(tf.__version__)
tf.enable_eager_execution() #For tf 1.x, for tf 2 you don't need this
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

1.13.1


## Use the IMDB dataset

In [4]:
imdb, info = tfds.load("imdb_reviews", with_info= True, as_supervised= True)
#tfds.list_builders() #to know all the datasets contained on tfds

train_data, test_data = imdb['train'], imdb['test']

### Create empty lists: 
training_sentences= []
training_labels= []

test_sentences= []
test_labels= []

for s,l in train_data: 
    training_sentences.append(str(s.numpy()))
    training_labels.append(str(l.numpy()))

for s,l in test_data:
    test_sentences.append(str(s.numpy()))
    test_labels.append(str(l.numpy()))


W0901 17:50:58.236824 4596544960 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


Instructions for updating:
Colocations handled automatically by placer.


W0901 17:50:58.327132 4596544960 deprecation.py:323] From /anaconda3/envs/Tensorflow/lib/python3.5/site-packages/tensorflow/python/data/ops/iterator_ops.py:532: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
print("Trainig sentences: \n",training_sentences[0], "\n")
print("Training labels: '0' is negative \n",training_labels[0], "\n")
print("Test sentences: \n", test_sentences[0], "\n")
print("Test labels: '1' is positive \n", test_labels[0], "\n")

Trainig sentences: 
 b'Sitting down to watch the 14th season of the Bachelor ("On the Wings of Love"), I knew I would be in for an "interesting" time. I had watched some of the previous seasons of the Bachelor in passing; watching an episode or two and missing the next three or so. I find that the Bachelor is often appealing and intriguing, though its quality and morality are often lacking.<br /><br />"On the Wings of Love" details the journey taken by Jake, a 31 year old commercial pilot from Dallas, Texas, to find true love, as true a love as one can find in a season-long reality-drama dating show. Jake meets 25 beautiful girls from all over the country. He begins to get to know them a bit, but it is mostly superficial; how well can you get to know someone in a few 5 minute conversations? Jake tries to make his true intentions known from the very beginning, at least to the audience. He noted that he doesn\'t just want love or a good time, but he wants a fianc\xc3\xa9 or wife. We can 

In [9]:
### When training my labels are expected to be numpy arrays: 
training_labels_final = np.array(training_labels)
test_labels_final= np.array(test_labels)

print("Training labels: \n", training_labels_final.shape, "\n")
print("Testing labels: \n", test_labels_final.shape, "\n")


Training labels: 
 (25000,) 

Testing labels: 
 (25000,) 



## Tokenize our sentences: 

In [16]:
### -- Hyperparameters:
vocab_size= 10000
embedding_dim= 16
max_length= 120
trunc_type= 'post'
oov_tok= "<OOV>"

tokenizer= Tokenizer(num_words= vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index= tokenizer.word_index
sequences= tokenizer.texts_to_sequences(training_sentences)
padded= pad_sequences(sequences=sequences, maxlen= max_length, truncating=trunc_type)

### -- Test sequences: 
testing_sequences= tokenizer.texts_to_sequences(test_sentences)
testing_padded= pad_sequences(testing_sequences, maxlen=max_length)

### --- Neural Network: 
#1) DEFINE THE MODEL: Sequential: defines a SEQUENCE of layers in the NN
# 1.1) Embedding layer: the key to text sentiment analysis in TF
# 1.2) Flatten:
# 1.3) Dense: fully connected neurons, first hidden layer of 6 neurons
# 1.4) Dense: one neuron

model= tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length= max_length), #the most important for NLP
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

